*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you need to create a machine learning model that will classify SMS messages as either "ham" or "spam". A "ham" message is a normal message sent by a friend. A "spam" message is an advertisement or a message sent by a company.

You should create a function called `predict_message` that takes a message string as an argument and returns a list. The first element in the list should be a number between zero and one that indicates the likeliness of "ham" (0) or "spam" (1). The second element in the list should be the word "ham" or "spam", depending on which is most likely.

For this challenge, you will use the [SMS Spam Collection dataset](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/). The dataset has already been grouped into train data and test data.

The first two cells import the libraries and data. The final cell tests your model and function. Add your code in between these cells.


In [1]:
# import libraries
#try:
  # %tensorflow_version only exists in Colab.
#  !pip install tf-nightly
#except Exception:
#  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
from keras.preprocessing import sequence

#!pip install tensorflow-datasets
#import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

2.4.1


In [2]:
# get data files
#!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
#!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

#train_file_path = "train-data.tsv"
#test_file_path = "valid-data.tsv"

In [15]:
train_data = pd.read_csv("train-data.tsv", sep='\t', header=None)
test_data = pd.read_csv("valid-data.tsv", sep='\t', header=None)

In [16]:
train_data[0] = train_data[0].astype("category")
train_data[0] = train_data[0].cat.codes
test_data[0] = test_data[0].astype("category")
test_data[0] = test_data[0].cat.codes

In [5]:
train_labels = train_data[0]
test_labels = test_data[0]
train_data = train_data[1]
test_data = test_data[1]
#train_data = tf.keras.preprocessing.sequence.pad_sequences(train_data,maxlen=250)
#test_data = sequence.pad_sequences(test_data, 250)

In [6]:
max_len = 250

In [7]:
train_data = np.array(train_data)

encoder = tf.keras.layers.experimental.preprocessing.TextVectorization(
 output_mode='int',
 output_sequence_length=max_len)

encoder.adapt(train_data)


In [8]:
model = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=64,
        # Use masking to handle the variable sequence lengths
        mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)
])

model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

In [9]:
history = model.fit(train_data, train_labels, epochs=10)

Epoch 1/10
131/131 [==============================] - 50s 274ms/step - loss: 0.6662 - accuracy: 0.8695
Epoch 2/10
131/131 [==============================] - 48s 366ms/step - loss: 0.4193 - accuracy: 0.8611
Epoch 3/10
131/131 [==============================] - 52s 398ms/step - loss: 0.1681 - accuracy: 0.8972
Epoch 4/10
131/131 [==============================] - 49s 377ms/step - loss: 0.0647 - accuracy: 0.9871
Epoch 5/10
131/131 [==============================] - 51s 388ms/step - loss: 0.0426 - accuracy: 0.9915
Epoch 6/10
131/131 [==============================] - 52s 395ms/step - loss: 0.0263 - accuracy: 0.9935
Epoch 7/10
131/131 [==============================] - 47s 356ms/step - loss: 0.0204 - accuracy: 0.9959
Epoch 8/10
131/131 [==============================] - 45s 341ms/step - loss: 0.0140 - accuracy: 0.9980
Epoch 9/10
131/131 [==============================] - 48s 368ms/step - loss: 0.0084 - accuracy: 0.9993
Epoch 10/10
131/131 [==============================] - 51s 392ms/step - l

In [19]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  prediction = model.predict(np.array([pred_text]))
  if prediction[0] >= 0.0 :
      prediction_list = [prediction[0], "spam"]
  else : 
      prediction_list = [prediction[0], "ham"]


  return (prediction_list)

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

[array([-6.813062], dtype=float32), 'ham']


In [20]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


You passed the challenge. Great job!
